In [2]:
import numpy as np
import pandas as pd
import os
import numpy as np
import pandas as pd
import os
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import time



In [3]:
start = time.time()


dataset= pd.read_csv("./news_articles.csv")
dataset

,docno,text
0,FBIS3-40266,languag romanian articl typecso text draft law...
1,FBIS3-40267,languag romanian articl typecso articl stelian...
2,FBIS3-40268,languag slovak articl typebfn commentari juliu...
3,FBIS3-40269,languag slovak articl typecso interview pavol ...
4,FBIS3-40270,languag slovak articl typebfn ta sr report ste...
...,...,...
414950,LA040890-0022,regard april 1 pop eye report film secur guard...
414951,LA040890-0023,mark leviton whose march 25 letter respond dvo...
414952,LA040890-0024,true capit demand human interact conduct volun...
414953,LA040890-0025,with paint face depict classic work art van go...


In [4]:
%%time
dataset['text_list']= dataset['text'].apply(lambda x: [item for item in str(x).split()])
dataset

CPU times: user 9.01 s, sys: 11.6 s, total: 20.6 s
Wall time: 28.3 s


,docno,text,text_list
0,FBIS3-40266,languag romanian articl typecso text draft law...,"[languag, romanian, articl, typecso, text, dra..."
1,FBIS3-40267,languag romanian articl typecso articl stelian...,"[languag, romanian, articl, typecso, articl, s..."
2,FBIS3-40268,languag slovak articl typebfn commentari juliu...,"[languag, slovak, articl, typebfn, commentari,..."
3,FBIS3-40269,languag slovak articl typecso interview pavol ...,"[languag, slovak, articl, typecso, interview, ..."
4,FBIS3-40270,languag slovak articl typebfn ta sr report ste...,"[languag, slovak, articl, typebfn, ta, sr, rep..."
...,...,...,...
414950,LA040890-0022,regard april 1 pop eye report film secur guard...,"[regard, april, 1, pop, eye, report, film, sec..."
414951,LA040890-0023,mark leviton whose march 25 letter respond dvo...,"[mark, leviton, whose, march, 25, letter, resp..."
414952,LA040890-0024,true capit demand human interact conduct volun...,"[true, capit, demand, human, interact, conduct..."
414953,LA040890-0025,with paint face depict classic work art van go...,"[with, paint, face, depict, classic, work, art..."


In [5]:
%%time
model = Word2Vec(sentences=dataset['text_list'], vector_size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")
#model.train([["hello", "world"]], total_examples=1, epochs=1)
model.init_sims(replace = True)

<timed exec>:5: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).


CPU times: user 22min 23s, sys: 55.2 s, total: 23min 18s
Wall time: 7min 1s


In [6]:
%%time


def vectorize(list_of_docs, model):
    
    features = []

    for tokens in list_of_docs:
        zero_vector = np.zeros(model.vector_size)
        vectors = []
        for token in tokens:
            if token in model.wv:
                try:
                    vectors.append(model.wv[token])
                except KeyError:
                    continue
        if vectors:
            vectors = np.asarray(vectors)
            avg_vec = vectors.mean(axis=0)
            features.append(avg_vec)
        else:
            features.append(zero_vector)
    return features
    
vectorized_docs = vectorize(dataset['text_list'], model=model)
len(vectorized_docs), len(vectorized_docs[0])

CPU times: user 2min 22s, sys: 3.49 s, total: 2min 25s
Wall time: 2min 29s


(414955, 100)

In [7]:
vectorized_docs[0]


array([-0.00294398, -0.01384749, -0.0040934 ,  0.00838549, -0.05671015,
        0.04184359, -0.04622349, -0.03068027,  0.05583195, -0.02877771,
        0.01134385,  0.0440376 , -0.04862376,  0.010643  , -0.00150023,
       -0.0287356 ,  0.01880686,  0.02324464,  0.01030319,  0.02078049,
       -0.04887428,  0.04620403, -0.07223277,  0.06690155,  0.02653569,
       -0.00860831, -0.05414249,  0.0449677 , -0.03837787, -0.03090131,
       -0.05174911, -0.01519096, -0.00951495,  0.00776517, -0.02213947,
       -0.00983816,  0.00708874,  0.01926752,  0.01082456,  0.0085567 ,
       -0.01735372,  0.03565613, -0.029776  ,  0.0356724 , -0.07297955,
        0.06803109, -0.04615786, -0.04392191,  0.00098912, -0.10658194,
        0.02832556,  0.01755891,  0.06039837, -0.01472596,  0.01327457,
       -0.04809245,  0.01069252, -0.04869824,  0.04393815, -0.00598279,
        0.05546375,  0.03645775, -0.05097708,  0.0062453 ,  0.0003104 ,
       -0.02170113, -0.03419081,  0.04173635,  0.08032178, -0.03

In [8]:
end = time.time()
print(end - start)

604.7363240718842


In [9]:
new_model = Word2Vec.load('word2vec.model')
print(new_model)

Word2Vec<vocab=1189200, vector_size=100, alpha=0.025>


Pipeline 2 for Reduced data set

In [17]:
reduced_data_set=pd.read_csv("./news_data.csv")
reduced_data_set


,docno,doclen,text
0,LA071790-0001,318,for prefer skip museum go directli gift shop n...
1,LA062989-0001,44,express conflict ideolog even contain symbol s...
2,LA062989-0002,51,how inan reason five suprem court justic rule ...
3,LA062989-0003,31,former california chief justic rose bird use m...
4,LA062989-0004,117,angri congressmen vow protect flag headlin art...
...,...,...,...
20567,FT922-1326,106,anglia televis announc costcut measur includ d...
20568,FT922-1327,254,compani are encourag employe travel road rail ...
20569,FT922-1328,64,the shambl built 13th centuri citi butcheri ce...
20570,FT922-1329,115,mr andrew kimmin jail head fail blade group fu...


In [19]:
reduced_data_set['text_list']= reduced_data_set['text'].apply(lambda x: [item for item in str(x).split()])
reduced_data_set

,docno,doclen,text,text_list
0,LA071790-0001,318,for prefer skip museum go directli gift shop n...,"[for, prefer, skip, museum, go, directli, gift..."
1,LA062989-0001,44,express conflict ideolog even contain symbol s...,"[express, conflict, ideolog, even, contain, sy..."
2,LA062989-0002,51,how inan reason five suprem court justic rule ...,"[how, inan, reason, five, suprem, court, justi..."
3,LA062989-0003,31,former california chief justic rose bird use m...,"[former, california, chief, justic, rose, bird..."
4,LA062989-0004,117,angri congressmen vow protect flag headlin art...,"[angri, congressmen, vow, protect, flag, headl..."
...,...,...,...,...
20567,FT922-1326,106,anglia televis announc costcut measur includ d...,"[anglia, televis, announc, costcut, measur, in..."
20568,FT922-1327,254,compani are encourag employe travel road rail ...,"[compani, are, encourag, employe, travel, road..."
20569,FT922-1328,64,the shambl built 13th centuri citi butcheri ce...,"[the, shambl, built, 13th, centuri, citi, butc..."
20570,FT922-1329,115,mr andrew kimmin jail head fail blade group fu...,"[mr, andrew, kimmin, jail, head, fail, blade, ..."


In [20]:
%%time
reduced_model = Word2Vec(sentences=reduced_data_set['text_list'], vector_size=100, window=5, min_count=1, workers=4)
reduced_model.save("word2vec_reduced.model")
reduced_model = Word2Vec.load("word2vec_reduced.model")
#model.train([["hello", "world"]], total_examples=1, epochs=1)


CPU times: user 44.1 s, sys: 583 ms, total: 44.7 s
Wall time: 13.5 s


In [21]:

reduced_vectorized_docs = vectorize(reduced_data_set['text_list'], model=reduced_model)
len(reduced_vectorized_docs), len(reduced_vectorized_docs[0])

(20572, 100)

In [25]:
reduced_vectorized_docs[0]


array([-0.4988688 ,  0.21900398, -0.16708075, -0.0242645 ,  0.04169853,
       -0.26426315, -0.27457842, -0.14144413, -0.01070359, -0.375529  ,
       -0.05766171, -0.2198541 , -0.4031222 ,  0.24664193, -0.1666716 ,
        0.18219598, -0.09216399, -0.7620696 , -0.20601214, -0.2824426 ,
        0.22303241,  0.28608206, -0.2025756 , -0.22865717,  0.32677898,
        0.2584502 ,  0.03104377, -0.2126284 , -0.07139192,  0.52854586,
        0.19447397, -0.5787128 , -0.16183366, -0.5143431 , -0.41269627,
        0.17246288, -0.3170032 ,  0.05166046,  0.31478143, -0.94841903,
        0.48832294, -0.5323324 , -1.0124422 ,  0.11877174,  0.9550028 ,
        0.11134574,  0.37900946,  0.05207527,  0.39336413, -0.01033391,
        0.37967804,  0.15098879,  0.3336518 ,  0.05133928, -0.19323045,
        0.36210895, -0.35816887,  0.11891456, -0.5706752 , -0.12346888,
       -0.20835486,  0.538059  , -0.11818978, -0.4368812 , -0.21630977,
        0.47796032, -0.21824273, -0.16380493,  0.17581405, -0.18

In [43]:
dataframe=pd.DataFrame(reduced_vectorized_docs)
dataframe['docno']=reduced_data_set['docno']
dataframe.set_index('docno')
dataframe.to_csv('Reduced_Features_W2vv.csv',index=False)

In [44]:
re=pd.read_csv('./Reduced_Features_W2vv.csv',index_col='docno')
re

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
docno,,,,,,,,,,,,,,,,,,,,,
LA071790-0001,-0.498869,0.219004,-0.167081,-0.024264,0.041699,-0.264263,-0.274578,-0.141444,-0.010704,-0.375529,...,0.169272,0.118060,-0.024344,-0.120058,-0.200116,-0.225242,0.428010,-0.025674,-0.481440,0.214447
LA062989-0001,-0.517026,0.276076,-0.048357,0.430875,-0.257898,-0.538107,0.271947,0.465942,0.097223,-0.505209,...,0.305990,0.232186,-0.121830,-0.266491,-0.771098,-0.565878,0.550054,0.122493,-0.107015,0.559798
LA062989-0002,-0.546283,0.746436,0.063966,0.385783,0.007986,-0.159829,-0.165566,0.338764,0.014240,-0.201551,...,0.156223,-0.016663,-0.040424,-0.052968,-0.387806,-0.384883,0.482937,0.293252,-0.140836,0.252619
LA062989-0003,-0.451571,0.854950,-0.140325,0.377910,0.275739,-0.128421,0.041172,0.140137,0.124600,-0.721244,...,0.058879,-0.266949,-0.166727,-0.265439,-0.231247,-0.306825,0.636286,0.182501,-0.558871,0.031227
LA062989-0004,-0.607108,0.585273,-0.313340,0.440334,-0.255223,-0.239435,0.002132,0.136202,0.277858,-0.486047,...,0.133684,0.145780,-0.211357,-0.012548,-0.800308,-0.794123,0.753373,0.651380,-0.451873,0.566203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FT922-1326,-0.778417,0.035509,-0.563502,0.275048,0.080729,-0.629866,-0.505339,-0.299923,0.047976,-1.056282,...,0.554853,-0.068133,0.219019,-0.119115,-0.420004,-0.461077,0.571231,-0.052342,-0.746039,0.507499
FT922-1327,-0.833300,0.225373,-0.427662,0.458126,0.045744,-0.587477,-0.816268,-0.041575,0.074463,-1.003994,...,0.930983,0.080159,0.004781,-0.154427,0.276763,-0.132863,0.263154,-0.025363,-1.216235,0.632720
FT922-1328,-0.284822,0.110813,-0.124406,-0.160506,-0.214762,-0.091233,-0.186311,0.069555,-0.069211,-0.428160,...,0.158726,0.132019,-0.154628,-0.143778,0.215180,-0.096192,0.247957,-0.301821,-0.413313,0.264234
